# Lesson 4: Evaluate the Tuned Model

#### Project environment setup

- Install Tensorboard (if running locally)
```Python
!pip install tensorboard
```

In [ ]:
!pip install tensorboard

In [1]:
import os
import json
import re
import sys
import pandas as pd

custom_module_dir = 'C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/modules'
sys.path.append(custom_module_dir)

from utils_2 import print_d

### Explore results with Tensorboard

In [2]:
%load_ext tensorboard

In [3]:
%ls "C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/logs/"

 Volume in drive C has no label.
 Volume Serial Number is AEA6-CDE7

 Directory of C:\Users\MD. REZUWAN HASAN\Desktop\Jupyter Notebooks\rlhf\logs

09/06/2024  04:42 PM    <DIR>          .
09/06/2024  04:42 PM    <DIR>          ..
09/06/2024  04:41 PM    <DIR>          reinforcer-fulldata-logs
09/06/2024  04:42 PM    <DIR>          reinforcer-logs
09/06/2024  04:40 PM    <DIR>          reward-logs
               0 File(s)              0 bytes
               5 Dir(s)  31,049,379,840 bytes free


In [4]:
reward_logs = "C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/logs/reward-logs/"
reinforcer_logs = "C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/logs/reinforcer-logs/"
reinforcer_fulldata_logs = "C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/logs/reinforcer-fulldata-logs/"

port = "8000"
port2 = "8001"
port3 = "8002"

In [5]:
%tensorboard --logdir 'C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/logs' --bind_all 

Reusing TensorBoard on port 6009 (pid 20100), started 2:17:03 ago. (Use '!kill 20100' to kill it.)

- The dictionary of 'parameter_values' defined in the previous lesson

In [6]:
parameter_values={
        "preference_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/summarize_from_feedback_tfds/comparisons/train/*.jsonl",
        "prompt_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/reddit_tfds/train/*.jsonl",
        "eval_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/reddit_tfds/val/*.jsonl",
        "large_model_reference": "llama-2-7b",
        "reward_model_train_steps": 1410,
        "reinforcement_learning_train_steps": 320,
        "reward_model_learning_rate_multiplier": 1.0,
        "reinforcement_learning_rate_multiplier": 1.0,
        "kl_coeff": 0.1,
        "instruction":\
    "Summarize in less than 50 words"}

**Note:** Here, we are using "text_small" for our datasets for learning purposes. However for the results that we're evaluating in this lesson, the team used the full dataset with the following hyperparameters:

```Python
parameter_values={
        "preference_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/summarize_from_feedback_tfds/comparisons/train/*.jsonl",
        "prompt_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/reddit_tfds/train/*.jsonl",
        "eval_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/reddit_tfds/val/*.jsonl",
        "large_model_reference": "llama-2-7b",
        "reward_model_train_steps": 10000,
        "reinforcement_learning_train_steps": 10000, 
        "reward_model_learning_rate_multiplier": 1.0,
        "reinforcement_learning_rate_multiplier": 0.2,
        "kl_coeff": 0.1,
        "instruction":\
    "Summarize in less than 50 words"}
```

# Evaluate using the tuned and untuned model

In [7]:
dataset_path = 'C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/data'

eval_tuned_path = f'{dataset_path}/eval_results_tuned.jsonl'
eval_data_tuned = []

eval_untuned_path = f'{dataset_path}/eval_results_untuned.jsonl'
eval_data_untuned = []

## Tuned path

In [8]:
with open(eval_tuned_path) as f:
    for line in f:
        eval_data_tuned.append(json.loads(line))

In [9]:
eval_data_tuned

[{'inputs': {'inputs_pretokenized': "Summarize in less than 50 words.\n\n\nBefore anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: ",
   'targets_pretokenized': ''},
  'prediction': "My country's equivalent to Valentine's Day is coming. Want to surprise my girlfriend with roses but don't know if she would like getting some. Any ideas on how to get that information out of her without spoiling the surprise"},
 {'inputs': {'inputs_pretokenized': 'Summarize in less than 50 words.\n\n\nFor most of high sch

In [10]:
len(eval_data_tuned)

5

In [11]:
eval_data_tuned[0]

{'inputs': {'inputs_pretokenized': "Summarize in less than 50 words.\n\n\nBefore anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: ",
  'targets_pretokenized': ''},
 'prediction': "My country's equivalent to Valentine's Day is coming. Want to surprise my girlfriend with roses but don't know if she would like getting some. Any ideas on how to get that information out of her without spoiling the surprise"}

In [12]:
print_d(eval_data_tuned[0]) # Look at the result produced by the tuned model

--------------------------------------------------
key:inputs

val
  --------------------------------------------------
  key:inputs_pretokenized

  val:Summarize in less than 50 words.


Before anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: 
  --------------------------------------------------
  key:targets_pretokenized

  val:
--------------------------------------------------
key:prediction

val:My country's equivalent to Valentine's Day is coming. Want to surprise my girlfriend with roses but do

## Untuned path

In [13]:
with open(eval_untuned_path) as f:
    for line in f:
        eval_data_untuned.append(json.loads(line))

In [14]:
eval_data_untuned

[{'inputs': {'inputs_pretokenized': "Summarize in less than 50 words.\n\n\nBefore anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: ",
   'targets_pretokenized': ''},
  'prediction': "The author wants to surprise his girlfriend with roses on Valentine's Day but he doesn't know if she likes roses. He wants to find out without spoiling the surprise."},
 {'inputs': {'inputs_pretokenized': 'Summarize in less than 50 words.\n\n\nFor most of high school, I\'ve been the go to "computer kid." I\'ll be the firs

In [15]:
len(eval_data_untuned)

5

In [16]:
eval_data_untuned[0]

{'inputs': {'inputs_pretokenized': "Summarize in less than 50 words.\n\n\nBefore anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: ",
  'targets_pretokenized': ''},
 'prediction': "The author wants to surprise his girlfriend with roses on Valentine's Day but he doesn't know if she likes roses. He wants to find out without spoiling the surprise."}

In [17]:
# Look at the result produced by the untuned model
print_d(eval_data_untuned[0])

--------------------------------------------------
key:inputs

val
  --------------------------------------------------
  key:inputs_pretokenized

  val:Summarize in less than 50 words.


Before anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: 
  --------------------------------------------------
  key:targets_pretokenized

  val:
--------------------------------------------------
key:prediction

val:The author wants to surprise his girlfriend with roses on Valentine's Day but he doesn't know if she l

### Explore the results side by side in a dataframe

In [18]:
# Extract all the prompts
prompts = [sample['inputs']['inputs_pretokenized']
           for sample in eval_data_tuned]

In [19]:
prompts

["Summarize in less than 50 words.\n\n\nBefore anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: ",
 'Summarize in less than 50 words.\n\n\nFor most of high school, I\'ve been the go to "computer kid." I\'ll be the first to admit that I know a lot about how computers work, and often fix things for teachers before the IT guys have a chance to get a whack at them. I worked at a computer repair shop for half a year as a technician. I\'ve done the typical tech guy thing and drooled over new computers that 

In [20]:
prompts[0]

"Summarize in less than 50 words.\n\n\nBefore anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]: "

In [21]:
# Completions from the untuned model
untuned_completions = [sample['prediction']
                       for sample in eval_data_untuned]

In [22]:
untuned_completions

["The author wants to surprise his girlfriend with roses on Valentine's Day but he doesn't know if she likes roses. He wants to find out without spoiling the surprise.",
 'A high school senior who is going to major in Computer Network Information Security and Digital Forensics needs a laptop that is powerful enough to run virtual desktops. He is open to suggestions of any brands, but prefers Macs. He is looking for a laptop under $1600 and wants to know the difference between MacBook and',
 "Graduated in May 2014, but couldn't start applying to jobs till April of this year. Applied to nearly 50 places, but only got 2 interviews. GPA is higher than 3 but lower than 3.5. Don't know what I'm doing wrong.",
 'Graduated a few months ago, looking for a job in marketing/sales. Found a small local company that is successful but has an awful website. I want to apply for a marketing job but not sure if they are hiring. I thought of building a dummy website for them and attach some screenshots to

In [23]:
untuned_completions[0]

"The author wants to surprise his girlfriend with roses on Valentine's Day but he doesn't know if she likes roses. He wants to find out without spoiling the surprise."

In [24]:
# Completions from the tuned model
tuned_completions = [sample['prediction']
                     for sample in eval_data_tuned]

In [25]:
tuned_completions

["My country's equivalent to Valentine's Day is coming. Want to surprise my girlfriend with roses but don't know if she would like getting some. Any ideas on how to get that information out of her without spoiling the surprise",
 'Been the go to "computer kid" in high school. Majoring in Computer Network Information Security and Digital Forensics. Need suggestions for a laptop under 1600. What are the big differences between a MacBook and MacBook Pro',
 "Graduated in may 2014. Applied to nearly 50+ places for data, analysis finance entry level. Barely get any response. Don't know what to do next. Would appreciate advice from recent econ/finance grads or hiring personnel.",
 'Graduated a few months ago. No luck so far to get a job in marketing/sales. Thought to build a dummy website fitted to the company and attach some screenshots to my application. What do you think?',
 'About 10 years ago applied for my first credit card AMEX delta card. No longer have time to travel as much. Thinkin

In [26]:
tuned_completions[0]

"My country's equivalent to Valentine's Day is coming. Want to surprise my girlfriend with roses but don't know if she would like getting some. Any ideas on how to get that information out of her without spoiling the surprise"

- Now putting all together in one big dataframe

In [27]:
results = pd.DataFrame(
    data={'prompt': prompts,
          'base_model':untuned_completions,
          'tuned_model': tuned_completions})

In [28]:
pd.set_option('display.max_colwidth', None)

In [29]:
# Print the results
results

,prompt,base_model,tuned_model
0,"Summarize in less than 50 words.\n\n\nBefore anything, not a sad story or anything. My country's equivalent to Valentine's Day is coming and I had this pretty simple idea to surprise my girlfriend and it would involve giving her some roses. The thing is, although I know she would appreciate my intention in and of itself, I don't know if she would like the actual flowers and such, so I wanted to find out if she likes roses and if she would like getting some, but without her realizing it so as not to spoil the surprise. Any ideas on how to get that information out of her? [summary]:",The author wants to surprise his girlfriend with roses on Valentine's Day but he doesn't know if she likes roses. He wants to find out without spoiling the surprise.,My country's equivalent to Valentine's Day is coming. Want to surprise my girlfriend with roses but don't know if she would like getting some. Any ideas on how to get that information out of her without spoiling the surprise
1,"Summarize in less than 50 words.\n\n\nFor most of high school, I've been the go to ""computer kid."" I'll be the first to admit that I know a lot about how computers work, and often fix things for teachers before the IT guys have a chance to get a whack at them. I worked at a computer repair shop for half a year as a technician. I've done the typical tech guy thing and drooled over new computers that come out (read: Wired/PopSci/2600 articles and centerfolds). \n\nThat said, I have NO IDEA what kind of computer to get for college. I've always wanted to buy a ton of parts and build my own ""super"" desktop, but I for sure need a laptop. If everything goes according to plan, I'm enrolling at Champlain College next Fall to double major in Computer Network Information Security and Digital Forensics. \n\nA lot of you probably already know this, but this major involves a lot of virtual desktops. Essentially I set up entire networks on my own rig and then go in and play around with them, all from one computer. It takes a lot of processing power and hard drive space.\n\nOver the years I've built up several stereotypes about different brands, operating systems, products... Recently, my mind has been opened back up and I'm willing to take a look at anything, even a Dell or an Asus.\n\nIt's recently been brought to my attention that Macs are actually pretty good. I've always been a notorious Windows-only-guy. I finally decided to bite my tongue and take a look under the hood, and my god are these things powerful... I guess it really does help making everything but the processor in-house.\n\nAnywho, what I really need is suggestions that might work well for my major, hopefully staying under a budget of $1600. Also, what are the big differences between a MacBook and MacBook Pro? [summary]:","A high school senior who is going to major in Computer Network Information Security and Digital Forensics needs a laptop that is powerful enough to run virtual desktops. He is open to suggestions of any brands, but prefers Macs. He is looking for a laptop under $1600 and wants to know the difference between MacBook and","Been the go to ""computer kid"" in high school. Majoring in Computer Network Information Security and Digital Forensics. Need suggestions for a laptop under 1600. What are the big differences between a MacBook and MacBook Pro"
2,"Summarize in less than 50 words.\n\n\nSo I graduated in may 2014 in the DC area but couldn't start applying to jobs till April of this year due to family circumstances. So far I've applied to nearly 50+ places, 4 recruiters contacted me and added me to their available candidates list after a small interview (one of them got me a small office assignment but that's over) and gotten two interviews by myself (none of which panned out). I have experience in office software and am pretty well versed in excel (i'd say a little less than advanced but a lot more than proficient) and advanced mathematics. I have a GPA high

In [30]:
results.to_excel("C:/Users/MD. REZUWAN HASAN/Desktop/Jupyter Notebooks/rlhf/outputs/results.xlsx",index = False)